In [1]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
from pathlib import Path
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install --no-index --find-links ../input/huggingface-datasets datasets -q

In [3]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [4]:
import pandas as pd
df = pd.read_csv(path/'train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [5]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [6]:
df['input'] = 'Text1: ' + df.context + '; Text2: ' + df.target + '; Anc1: ' + df.anchor

In [7]:
df.input.head()

0    Text1: A47; Text2: abatement of pollution; Anc...
1    Text1: A47; Text2: act of abating; Anc1: abate...
2    Text1: A47; Text2: active catalyst; Anc1: abat...
3    Text1: A47; Text2: eliminating process; Anc1: ...
4    Text1: A47; Text2: forest region; Anc1: abatement
Name: input, dtype: object

In [8]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

In [9]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [10]:
model_nm = '../input/debertav3small'

In [14]:
from transformers import AutoModelForSequenceClassification,DebertaV2Tokenizer
tokz = DebertaV2Tokenizer.from_pretrained(model_nm, use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def tokenization_func(x):
    return tokz(x["input"])

In [16]:
tokenized_ds = ds.map(tokenization_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [17]:
tokenized_ds[0]

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'Text1: A47; Text2: abatement of pollution; Anc1: abatement',
 'input_ids': [1,
  7655,
  435,
  294,
  336,
  5753,
  346,
  7655,
  445,
  294,
  47284,
  265,
  6435,
  346,
  816,
  1207,
  435,
  294,
  47284,
  2],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [18]:
tokenized_ds = tokenized_ds.rename_columns({'score': 'labels'})

In [19]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [20]:
dds = tokenized_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [21]:
eval_df['input'] = 'Text1: ' + eval_df.context + '; Text2: ' + eval_df.target + '; Anc1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tokenization_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
import numpy as np
def corr(x, y):
    return np.corrcoef(x, y)[0][1]

def corr_d(eval_pred):
    return {
        'pearson': corr(*eval_pred)
    }

In [23]:
from transformers import TrainingArguments, Trainer

In [24]:
bs = 128
epochs = 5
lr = 1e-5

In [25]:
args = TrainingArguments('outputs', 
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,
                         report_to='none')

In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model,
                  args,
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=corr_d)

Some weights of the model checkpoint at ../input/debertav3small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, context, id, target, anchor. If input, context, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 10
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2140


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.043361,0.707862
2,No log,0.028573,0.785559
3,0.061600,0.028803,0.802880
4,0.061600,0.028190,0.811252
5,0.024900,0.024114,0.814953
6,0.024900,0.024207,0.819655
7,0.024900,0.024281,0.819587
8,0.020600,0.026097,0.820859
9,0.020600,0.025019,0.821259
10,0.018700,0.025044,0.821407


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, context, id, target, anchor. If input, context, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, context, id, target, anchor. If input, context, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved 

TrainOutput(global_step=2140, training_loss=0.03062073437967033, metrics={'train_runtime': 712.6446, 'train_samples_per_second': 383.838, 'train_steps_per_second': 3.003, 'total_flos': 1863117245289900.0, 'train_loss': 0.03062073437967033, 'epoch': 10.0})

In [28]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds = np.clip(preds, 0, 1)

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: context, input, id, target, anchor. If context, input, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


In [29]:
import datasets
submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1061